In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [3]:
phewas_all <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp/ukb24983_imp_v3.glm.logistic.hybrid.gz'


In [24]:
phewas_hits <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp/ukb24983_imp_v3.phewas.hits.tsv'


In [4]:
phewas_df <- fread(cmd=paste('zcat', phewas_all)) %>% 
rename('CHROM' = '#CHROM')

In [7]:
phewas_df %>% select(GBE_ID) %>% unique() %>% pull() %>% length()

[1] 172

In [21]:
phewas_df %>% 
filter(
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)
) %>%
select(ID) %>% unique() %>% pull() %>% length()

[1] 43

In [22]:
0.05 / (172 * 43)

[1] 6.760411e-06

In [15]:
library(googlesheets)
gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")

file <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A'
GBE_names_df <- file %>% gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


In [25]:
phewas_hits_df <- phewas_df %>% filter(
    P < 5e-6,
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)    
)  %>%
select(-REF, -ALT, -A1, -'FIRTH?', -TEST, -ERRCODE) %>%
left_join(
    GBE_names_df %>% select(GBE_ID, GBE_short_name), 
    by='GBE_ID'
)

In [26]:
phewas_hits_df %>% dim()

[1] 19 10

In [28]:
phewas_hits_df %>% select(ID) %>% pull() %>% unique() %>% length()


[1] 9

In [29]:
phewas_hits_df %>% select(GBE_ID) %>% pull() %>% unique() %>% length()


[1] 15

In [27]:
phewas_hits_df

CHROM,POS,ID,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,GBE_ID,GBE_short_name
<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,152285861,1:152285861:G:A,307048,1.265330,0.01971920,11.93410,7.86192e-33,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor
1,152285861,1:152285861:G:A,337146,1.229250,0.02384610,8.65571,4.89842e-18,HC382,Asthma
1,152285861,1:152285861:G:A,337146,1.915950,0.03621370,17.95490,4.39627e-72,HC261,Eczema/dermatitis
1,156784982,1:156784982:T:C,337146,0.930486,0.01097840,-6.56273,5.28301e-11,HC219,Hypothyroidism/myxoedema
6,7231843,6:7231843:G:A,337146,0.873727,0.01735970,-7.77589,7.49185e-15,HC221,Diabetes
6,7231843,6:7231843:G:A,337146,0.901531,0.02085780,-4.96987,6.69973e-07,HC243,Inguinal hernia
7,127015083,7:127015083:G:A,307048,0.897173,0.02176810,-4.98467,6.20685e-07,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor
7,127015083,7:127015083:G:A,337146,1.191720,0.03626720,4.83621,1.32336e-06,HC25,Cataract
8,145158607,8:145158607:G:A,307048,1.066280,0.01190740,5.38925,7.07523e-08,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor


In [30]:
phewas_hits_df %>% fwrite(phewas_hits, sep='\t')